Import

In [107]:
# import
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import numpy as np
from scipy.ndimage import gaussian_filter
from matplotlib import pyplot as plt
import random
import cv2
from scipy import signal
import os
# from skimage.morphology import skeletonize, thin
import math
from PIL import Image, ImageOps
import time

Constants

In [108]:
# Constants

# BODY PANTS MASK SHADOW REFELCTION
template_rgb = np.array([[255,16,16],[0,0,255],[0,255,0]])
bpm = np.array([[100,100],[150,60],[50,110]])

Building Block Functions

In [109]:
def getFigure(PIL_image):
    # Input: Image imported from PIL
    # Output: 2D Numpy where 1 is amongus and 0 is transparent
    rgb = red.convert("RGBA")
    arr = np.array(rgb)
    #print("RGBA", arr.shape)
    area = np.zeros((rgb.size[1], rgb.size[0]), dtype="uint8")  # width, height for y, x im programming
    area[(arr != arr[0][0]).all(axis=2)] = 255  # that pixel looks like background
    return area

def makeDictFromLists(test_keys, test_values):
    test_keys = list(test_keys)
    test_values = list(test_values)
    res = {}
    for key in test_keys:
        for value in test_values:
            res[key] = value
            test_values.remove(value)
            break
    return res

def convertTransparentFromPIL(img, num):
    # Input: RGB Image
    # Output: Mark amongs with num, the rest is transparent
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)
    img = img.convert("RGBA")

    datas = img.getdata()
    r, g, b = datas[0][0:3]
    newData = []

    for item in datas:
        if item[0] == r and item[1] == g and item[2] == b:  # transparent
            newData.append((num, num, num, 0))
        else:  # num
            newData.append((num, num, num, 255))

    #print("NEWDATA UNIQUE", np.unique(newData, return_counts=True))
    img.putdata(newData)
    return img


def convertTransparentFromNumpy(arr):
    # Input: 2d numpy array of rgb image
    #       where 255 is for amongus
    #             0 is for transparent
    # Output: RGBA with transparent
    newData = []

    for row in arr:
        r = []
        for col in row:
            if col == 0:
                r.append((0, 0, 0, 0))
            else:
                r.append((col, col, col, 1))
        newData.append(r)
    img = Image.fromarray(np.array(newData))
    return img


def show(imgs):
    for img in imgs:
        plt.imshow(img)
        plt.show()
        
def countUniquePixels(img):
    x = np.array(img)
    x = x.reshape(-1,x.shape[-1])
    return np.unique(x,return_counts=True,axis=0)

def makeDictFromLists(test_keys, test_values):
    test_keys = list(test_keys)
    test_values = list(test_values)
    res = {}
    for key in test_keys:
        for value in test_values:
            res[key] = value
            test_values.remove(value)
            break
    return res

def getTopPixels(img,top):
    pixel, count = countUniquePixels(img)
    count = count.reshape(-1,1)
    #print(111,pixel,count)
    arr = np.concatenate((count,pixel),axis=1)
    arr = arr[arr[:, 0].argsort()]
    return arr[::-1][:top:]

def showTopPixels(img):
    img = img[None,::,1::]
    show([img])

def showPixels(img,xy):
    arr = np.array(img)
    ans = []
    inner = []
    for x,y in xy:
        inner.append(arr[x][y][:3:])
    ans.append(inner)
    print("ANS",ans)
    show([ans])

# Ax = b >>> A = bx'
# from:x, to:b
def getMatrix(x,b):
    x=np.array(x)
    b=np.array(b)
    b = b.reshape(-1,b.shape[-1]).T
    x = x.reshape(-1,x.shape[-1]).T
    #print("b",b)
    #print("x",x)
    matrix = np.matmul(b,np.linalg.pinv(x))
    return matrix

def getRGB(img,xy):
    arr = np.array(img)
    ans = []
    inner = []
    for x,y in xy:
        inner.append(arr[x][y][:3:])
    # reflection and shadow
    ans.append(inner)
    return ans

def isCloseToTemplate(rgb):
    rgb = np.array(rgb)
    r = np.array([255,16,16])
    g = np.array([0,255,0])
    b = np.array([0,0,255])
    threshold = 265
    return np.sum(np.abs(rgb - r)) <= threshold or np.sum(np.abs(rgb - g)) <= threshold or np.sum(np.abs(rgb - b)) <= threshold

def make000Transparent(img):
    x = np.array(img)
    b = np.ones((x.shape[0],x.shape[1] , 1), dtype="uint8") * 255
    z = np.concatenate((x,b), axis = 2)
    idx = (z==[0,0,0,255]).all(axis=2)
    z[idx] = [0,0,0,0]
    return z

def find_percent_transparent(image):
    if len(image[0][0]) < 4:
        return 0
    return 100 - np.average(image[:,:,3])*100/255

Sub Functions

In [118]:
def pasteOn(fg, bg, size_multiply, rotate_angle, flip, mirror, x, y):
    # Input: Details of pasting
    # Output: New image of bg pasted with fg
    bg = bg.copy()
    fg = fg.copy()
    width, height = bg.size
    base_width = width/10

    fw, fh = fg.size
    if flip:
        fg = ImageOps.flip(fg)
    if mirror:
        fg = ImageOps.mirror(fg)
    #print("After flip,mirror", np.unique(fg, return_counts=True))
    fg = fg.resize((int(base_width*size_multiply), int(base_width/fw*fh*size_multiply)), Image.Resampling.NEAREST)
    #print("After resize", np.unique(fg, return_counts=True))
    fg = fg.rotate(rotate_angle, Image.Resampling.NEAREST, expand=1)
    #print("After rotate", np.unique(fg, return_counts=True))

    #print(width, height, x, y)
    loc_x = int(x*width)
    loc_y = int(y*height)
    bg.paste(fg, (loc_x, loc_y), fg.convert('RGBA'))
    return bg

def pasteOn2(fg, bg, size_multiply, rotate_angle, flip, mirror, x, y):
    # Input: Details of pasting
    # Output: New image of bg pasted with fg
    bg = bg.copy()
    fg = fg.copy()
    #print(type(bg))
    if isinstance(bg, np.ndarray):
        bg = Image.fromarray(bg)

    width, height = bg.size
    base_width = width/10

    if isinstance(fg, np.ndarray):
        fg = Image.fromarray(fg)
    
    fw, fh = fg.size
    if flip:
        fg = ImageOps.flip(fg)
    if mirror:
        fg = ImageOps.mirror(fg)
    #print("After flip,mirror", np.unique(fg, return_counts=True))
    fg = fg.resize((int(base_width*size_multiply), int(base_width/fw*fh*size_multiply)), Image.Resampling.NEAREST)
    #print("After resize", np.unique(fg, return_counts=True))
    fg = fg.rotate(rotate_angle, Image.Resampling.NEAREST, expand=1)
    #print("After rotate", np.unique(fg, return_counts=True))

    #print(width, height, x, y)
    loc_x = int(x*width)
    loc_y = int(y*height)
    bg.paste(fg, (loc_x, loc_y), fg.convert('RGBA'))
    return bg

def isImgOverlap(img1, img2):
    # img1 is before
    # img2 is after
    arr1 = np.array(img1)
    arr2 = np.array(img2)
    cnt1 = np.unique(arr1, return_counts=True)
    cnt2 = np.unique(arr2, return_counts=True)
    dic1 = makeDictFromLists(cnt1[0], cnt1[1])
    dic2 = makeDictFromLists(cnt2[0], cnt2[1])
    for key in dic1:
        if key != 0 and key in dic2 and dic1[key] > dic2[key]:
            return True
    return False

def isImgOverlap2(img1, img2):
    # img1 is before
    # img2 is after
    arr1 = np.array(img1)
    arr2 = np.array(img2)
    cnt1 = np.unique(arr1, return_counts=True)
    cnt2 = np.unique(arr2, return_counts=True)
    dic1 = makeDictFromLists(cnt1[0], cnt1[1])
    dic2 = makeDictFromLists(cnt2[0], cnt2[1])
    for key in dic1:
        if key != 0 and key in dic2 and dic1[key] > dic2[key] + 7000:
            return True
    return False

def isInBoundary(fg, bg, sz, rot, flip, mirror, x, y):
    width, height = bg.size
    fwidth, fheight = fg.size
    cx = int(width / 2)
    cy = int(height / 2)
    thetha = rot

    rad = math.radians(thetha)
    base_width = width/10
    fg = fg.resize((int(base_width*sz), int(base_width/fwidth*fheight*sz)))
    fwidth, fheight = fg.size
    loc_x = int(x*width)
    loc_y = int(y*height)
    corners = [[loc_x, loc_y], [loc_x+fwidth, loc_y], [loc_x, loc_y+fheight], [loc_x+fwidth, loc_y+fheight]]
    newCorners = []
    for px, py in corners:
        new_px = cx + int(float(px-cx) * math.cos(rad) + float(py-cy) * math.sin(rad))
        new_py = cy + int(-(float(px-cx) * math.sin(rad)) + float(py-cy) * math.cos(rad))
        #print(px, py, new_px, new_py, width, height)
        if new_px > width or new_px < 0:
            return False
        if new_py > height or new_py < 0:
            return False
    return True

def isInBoundary2(fg, bg, sz, rot, flip, mirror, x, y):
    width, height = bg.size
    fwidth, fheight = fg.size
    cx = int(width / 2)
    cy = int(height / 2)
    thetha = rot

    rad = math.radians(thetha)
    base_width = width/10
    fg = fg.resize((int(base_width*sz), int(base_width/fwidth*fheight*sz)))
    fwidth, fheight = fg.size
    loc_x = int(x*width)
    loc_y = int(y*height)
    corners = [[loc_x, loc_y], [loc_x+fwidth, loc_y], [loc_x, loc_y+fheight], [loc_x+fwidth, loc_y+fheight]]
    newCorners = []
    for px, py in corners:
        new_px = cx + int(float(px-cx) * math.cos(rad) + float(py-cy) * math.sin(rad))
        new_py = cy + int(-(float(px-cx) * math.sin(rad)) + float(py-cy) * math.cos(rad))
        #print(px, py, new_px, new_py, width, height)
        thres = 25
        if new_px > width + thres or new_px < -thres:
            return False
        if new_py > height + thres or new_py < -thres:
            return False
    return True

def transferColor(template, color_img):
    new_img = np.array(template)[::,::,:3:]
    template_arr = np.array(template)[::,::,:3:]
    color_arr = np.array(color_img)
    color_arr = color_arr[::,::,:3:]
    trgb = np.array([[255,16,16],[0,0,255],[0,255,0]])
    crgb = getRGB(color_img,bpm)
    #print(trgb,crgb)
    A = getMatrix(trgb,crgb)
    #print("A",A)
    #print("color arr", color_arr)
    for i in range(len(new_img)):
        for j in range(len(new_img[i])):
            if isCloseToTemplate(template_arr[i][j]):
                new_img[i][j]= A @ template_arr[i][j]
    new_img = make000Transparent(new_img)
    return new_img

def randomPaste():
    # Output: Random variables for pasting fg on bg
    sz = 1 + (random.random())
    rot = 360 * random.random()
    flip = random.randrange(0, 2)
    mirror = random.randrange(0, 2)
    x = random.random()
    y = random.random()
    return sz, rot, flip, mirror, x, y

def randomPaste2():
    # Output: Random variables for pasting fg on bg
    sz = 1 + (random.random())
    rot = ( -0.5 + random.random() ) * 40
    flip = 0
    mirror = random.randrange(0, 2)
    x = random.random()
    y = random.random()
    return sz, rot, flip, mirror, x, y

def randomPaste3():
    # Output: Random variables for pasting fg on bg
    sz = 1 + (random.random())
    rot = ( -0.5 + random.random() ) * 40
    flip = 0
    mirror = random.randrange(0, 2)
    x = -0.1 + random.random() * 1.1
    y = -0.1 + random.random() * 1.1
    return sz, rot, flip, mirror, x, y

def random_square_background(dir_name,size=-1,N = 1,threshold_percent = 100):
    l = []
    bglist = os.listdir(dir_name)
    count_fail = 0
    while len(l) < N:
        if count_fail > max(20,2*N):
            raise Exception('Fail to generate background corresponding to param')
        image_name = np.random.choice(bglist)
        #print(image_name)
        if image_name[-4::] not in [".png",".jpg"]:
            continue
        #print(f'{dir_name}\\{image_name}')
        background = cv2.imread(f'{dir_name}\\{image_name}',cv2.IMREAD_UNCHANGED)
        if size == -1:
            size = np.random.randint(100,800)
        elif type(size) == tuple:
            if len(size) != 2:
                raise Exception('You can only use a tuple with two integers!')
            else:
                size = np.random.randint(size[0],size[1])
        

        if background.ndim == 2:
            height, width = background.shape
        else:
            height, width, nchannel = background.shape
        
        if height > size or width > size:
            size = min(height,width)
        
        x = np.random.randint(0,max(width-size+1,1))
        y = np.random.randint(0,max(height-size+1,1))

        patch = background[y:y+size,x:x+size]

        #print(find_percent_transparent(patch) , threshold_percent)
        if find_percent_transparent(patch) < threshold_percent:
            l.append(patch)
            count_fail = 0
        else:
            count_fail += 1

    if N == 1:
        return l[0]
    return l

Main Functions

In [111]:


def gen1(fg, bg, it):
    # Initiate label
    label = Image.new("RGB", (bg.size))
    scale = 255
    minus = scale//it
    img = bg
    for i in range(it):
        sz, rot, flip, mirror, x, y = randomPaste()
        img = pasteOn(fg, img, sz, 0, 0, 0, x, y)
        figureImg = convertTransparentFromPIL(red, scale-minus*i)
        label = pasteOn(figureImg, label, sz, 0, 0, 0, x, y)
    return img, label

def gen2(fg, bg, it):
    # Same Size, No rotate, No flip, No mirror, No overlap, No out of edge
    # Initiate label
    label = Image.new("RGB", (bg.size))
    scale = 255
    minus = scale//it
    img = bg
    cnt = 0
    for i in range(it):
        sz, rot, flip, mirror, x, y = randomPaste()
        figureImg = convertTransparentFromPIL(red, scale-minus*i)
        if not isInBoundary(figureImg, label, sz, rot, flip, mirror, x, y):
            #print("OUT OF BOUNDARY")
            continue
        labelnew = pasteOn(figureImg, label, sz, rot, flip, mirror, x, y)
        if isImgOverlap(label, labelnew):
            # print("OVERLAP")
            continue
        img = pasteOn(fg, img, sz, rot, flip, mirror, x, y)
        label = labelnew
        cnt += 1
    return img, label, cnt

def gen3(template_imgset, color_imgset, it):
    bg = random_square_background("bg")
    # Same Size, No rotate, No flip, No mirror, No overlap, No out of edge
    # Initiate label
    label_size = bg.shape[:2:]
    print(label_size)
    label = Image.new("RGB", label_size)
    scale = 255
    minus = scale//it
    img = bg
    cnt = 0
    for i in range(it):
        t1 = time.time()
        sz, rot, flip, mirror, x, y = randomPaste2()
        template = random.choice(template_imgset)
        color = random.choice(color_imgset)
        t8 = time.time()
        crewmate = transferColor(template,color)
        t7 = time.time()
        figureImg = convertTransparentFromPIL(crewmate, scale-minus*i)
        t2 = time.time()
        if not isInBoundary(figureImg, label, sz, rot, flip, mirror, x, y):
            #print("OUT OF BOUNDARY")
            continue
        t3 = time.time()
        labelnew = pasteOn2(figureImg, label, sz, rot, flip, mirror, x, y)
        t4 = time.time()
        if isImgOverlap(label, labelnew):
            # print("OVERLAP")
            continue
        t5 = time.time()
        img = pasteOn2(crewmate, img, sz, rot, flip, mirror, x, y)
        t6 = time.time()
        label = labelnew
        cnt += 1
        times = [t1,t8,t7,t2,t3,t4,t5,t6]
        [print(times[i+1]-times[i]) for i in range(len(times)-1)]
        print("===")
    return img, label, cnt

def gen4(crewpool_imgset, it):
    bg = random_square_background("bg")
    # Same Size, No rotate, No flip, No mirror, No overlap, No out of edge
    # Initiate label
    if isinstance(bg,np.ndarray):
        bg = Image.fromarray(bg)
    label_size = bg.size
    #print(label_size)
    label = Image.new("RGB", label_size)
    scale = 255
    minus = scale//it
    img = bg
    cnt = 0
    for i in range(it):
        sz, rot, flip, mirror, x, y = randomPaste2()
        crewmate = random.choice(crewpool_imgset)
        figureImg = convertTransparentFromPIL(crewmate, scale-minus*i)
        if not isInBoundary(figureImg, label, sz, rot, flip, mirror, x, y):
            #print("OUT OF BOUNDARY")
            continue
        labelnew = pasteOn2(figureImg, label, sz, rot, flip, mirror, x, y)
        if isImgOverlap(label, labelnew):
            # print("OVERLAP")
            continue
        img = pasteOn2(crewmate, img, sz, rot, flip, mirror, x, y)
        label = labelnew
        cnt += 1
    return img, label, cnt

def gen5(crewpool_imgset, it):
    bg = random_square_background("bg")
    # Same Size, No rotate, No flip, No mirror, No overlap, No out of edge
    # Initiate label
    if isinstance(bg,np.ndarray):
        bg = Image.fromarray(bg)
    label_size = bg.size
    #print(label_size)
    label = Image.new("RGB", label_size)
    scale = 255
    minus = scale//it
    img = bg
    cnt = 0
    for i in range(it):
        sz, rot, flip, mirror, x, y = randomPaste3()
        crewmate = random.choice(crewpool_imgset)
        figureImg = convertTransparentFromPIL(crewmate, scale-minus*i)
        if not isInBoundary2(figureImg, label, sz, rot, flip, mirror, x, y):
            #print("OUT OF BOUNDARY")
            continue
        labelnew = pasteOn2(figureImg, label, sz, rot, flip, mirror, x, y)
        if isImgOverlap2(label, labelnew):
            # print("OVERLAP")
            continue
        img = pasteOn2(crewmate, img, sz, rot, flip, mirror, x, y)
        label = labelnew
        cnt += 1
    return img, label, cnt

Execute Generation

In [112]:
# Import Images

def import_colors():
    # assign directory
    directory = 'crewmate'
    img_list = []
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and str(f)[-4::] in [".png",".jpg",".jpeg"]:
            img = Image.open(f)
            img_list.append(img)
    return img_list

def import_template():
    # assign directory
    directory = 'walking'
    img_list = []
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and str(f)[-4::] in [".png",".jpg",".jpeg"]:
            img = Image.open(f)
            img_list.append(img)
    return img_list

def import_crewpool():
    # assign directory
    directory = 'crewpool_noshadow'
    img_list = []
    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f) and str(f)[-4::] in [".png",".jpg",".jpeg"]:
            img = Image.open(f)
            img_list.append(img)
    return img_list

template_imgset = import_template()
color_imgset = import_colors()
crewpool_imgset = import_crewpool()

In [113]:
# GENERATE CREWPOOL
# n=0
# m=0
# for i in range(len(template_imgset)):
#     for j in range(len(color_imgset)):
#         temp = template_imgset[i]
#         col = color_imgset[j]
#         crewmate = transferColor(temp,col)
#         img = Image.fromarray(crewmate)
#         img.save("crewpool/img"+str(n)+str(m)+".png")
#         m+=1
#         print(m,"colors finished")
#     n+=1
#     print(n,"animations finished")
# print("All animations finished")

In [123]:

# Import Images

###### DONT TOUCH ANYTHING ABOVE THIS #####

# CHANGE PARAMETERS BELOW AND RUN
setNo = "set12"
fout = open((setNo+"/count.txt"), "w", buffering=1)
for i in range(600):
    tries = random.randint(3,20)
    img, lab, cnt = gen5(crewpool_imgset, tries)
    img = img.convert("RGB")
    img.save(setNo+"/img"+str(i)+".jpg")
    lab.save(setNo+"/label"+str(i)+".png")
    fout.write(str(i)+" "+str(cnt)+"\n")
    fout.flush()
    print(str(i)+" images done")
fout.close()

0 images done
1 images done
2 images done
3 images done
4 images done
5 images done
6 images done
7 images done
8 images done
9 images done
10 images done
11 images done
12 images done
13 images done
14 images done
15 images done
16 images done
17 images done
18 images done
19 images done
20 images done
21 images done
22 images done
23 images done
24 images done
25 images done
26 images done
27 images done
28 images done
29 images done
30 images done
31 images done
32 images done
33 images done
34 images done
35 images done
36 images done
37 images done
38 images done
39 images done
40 images done
41 images done
42 images done
43 images done
44 images done
45 images done
46 images done
47 images done
48 images done
49 images done
50 images done
51 images done
52 images done
53 images done
54 images done
55 images done
56 images done
57 images done
58 images done
59 images done
60 images done
61 images done
62 images done
63 images done
64 images done
65 images done
66 images done
67 im